In [1]:
import pandas as pd
import numpy as np
import sqlite3
import json
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
EXPERIMENT_NAME = 'P3856'
RUN_NAME = 'P3856_YHE211_1_Slot1-1_1_5104'

In [3]:
MZ_TOLERANCE_PPM = 5
MZ_TOLERANCE_PERCENT = MZ_TOLERANCE_PPM * 10**-4
SCAN_TOLERANCE = 2
RT_TOLERANCE = 1

##### 3DID parameters

In [4]:
# load the detected features
EXPERIMENT_DIR = '/data2/experiments/{}'.format(EXPERIMENT_NAME)
FEATURES_DIR = '{}/features-3did'.format(EXPERIMENT_DIR)
FEATURES_FILE = '{}/exp-{}-run-{}-features-3did.pkl'.format(FEATURES_DIR, EXPERIMENT_NAME, RUN_NAME)
FEATURES_DEDUP_FILE = '{}/exp-{}-run-{}-features-3did-dedup.pkl'.format(FEATURES_DIR, EXPERIMENT_NAME, RUN_NAME)
MZ_COLUMN_NAME = 'monoisotopic_mz'

In [5]:
features_df = pd.read_pickle(FEATURES_FILE)

##### PASEF parameters

In [6]:
print('there are {} features prior to de-dup'.format(len(features_df)))

there are 177134 features prior to de-dup


In [7]:
# features_df['mz'] = features_df.recalibrated_monoisotopic_mz  # shorthand to reduce verbosity
features_df['mz'] = features_df[MZ_COLUMN_NAME]  # shorthand to reduce verbosity
features_df['mz_ppm_tolerance'] = features_df.mz * MZ_TOLERANCE_PERCENT / 100
features_df['mz_lower'] = features_df.mz - features_df.mz_ppm_tolerance
features_df['mz_upper'] = features_df.mz + features_df.mz_ppm_tolerance
features_df['scan_lower'] = features_df.scan_apex - SCAN_TOLERANCE
features_df['scan_upper'] = features_df.scan_apex + SCAN_TOLERANCE
features_df['rt_lower'] = features_df.rt_apex - RT_TOLERANCE
features_df['rt_upper'] = features_df.rt_apex + RT_TOLERANCE
# features_df['composite_key'] = features_df.apply(lambda row: '{},{},{}'.format(row.feature_id, row.precursor_id, row.percolator_idx), axis=1)
features_df['composite_key'] = features_df.apply(lambda row: '{},{}'.format(row.feature_id, row.precursor_id), axis=1)

In [8]:
# remove these after we're finished
columns_to_drop_l = []
columns_to_drop_l.append('mz')
columns_to_drop_l.append('mz_ppm_tolerance')
columns_to_drop_l.append('mz_lower')
columns_to_drop_l.append('mz_upper')
columns_to_drop_l.append('scan_lower')
columns_to_drop_l.append('scan_upper')
columns_to_drop_l.append('rt_lower')
columns_to_drop_l.append('rt_upper')
columns_to_drop_l.append('composite_key')


In [9]:
features_df.sort_values(by=['intensity'], ascending=False, inplace=True)

In [10]:
# see if any detections have a duplicate - if so, find the dup with the highest intensity and keep it
keep_l = []
for row in features_df.itertuples():
    dup_df = features_df[(features_df.mz > row.mz_lower) & (features_df.mz < row.mz_upper) & (features_df.scan_apex > row.scan_lower) & (features_df.scan_apex < row.scan_upper) & (features_df.rt_apex > row.rt_lower) & (features_df.rt_apex < row.rt_upper)].copy()
    # group the dups by charge - take the most intense for each charge
    for group_name,group_df in dup_df.groupby(['charge'], as_index=False):
        keep_l.append(group_df.iloc[0].composite_key)


In [11]:
# remove any features that are not in the keep list
dedup_df = features_df[features_df.composite_key.isin(keep_l)].copy()

In [12]:
number_of_dups = len(features_df)-len(dedup_df)
print('removed {} duplicates ({}% of the original detections)'.format(number_of_dups, round(number_of_dups/len(features_df)*100)))
print('there are {} detected de-duplicated features'.format(len(dedup_df)))

removed 587 duplicates (0% of the original detections)
there are 176547 detected de-duplicated features


In [13]:
# remove the columns we added earlier
dedup_df.drop(columns_to_drop_l, axis=1, inplace=True)

In [14]:
dedup_df.sample(n=3)

,monoisotopic_mz,charge,intensity,intensity_full_rt_extent,scan_apex,scan_curve_fit,rt_apex,rt_curve_fit,precursor_id,envelope,feature_id,mono_adjusted,original_phr_error,candidate_phr_error,original_phr,monoisotopic_mass
0,753.342420,2,21330.0,6292,490.25,False,1675.39,True,16024,"((753.3617, 8737.00), (753.8555, 7196.00), (75...",1602401,False,0.0172935,None,0.823624,1504.670241
0,651.343369,2,51140.0,3488,723.36,True,1925.89,True,5015,"((651.3407, 22115.00), (651.8427, 19729.00), (...",501501,False,0.277054,2.68047,0.892109,1300.672139
0,879.436726,3,70325.0,4944,331.24,False,2095.77,False,32114,"((879.4338, 14065.00), (879.4338, 14065.00), (...",3211401,False,-0.299731,6.60402,1.000000,2635.288278


In [15]:
dedup_df.to_pickle(FEATURES_DEDUP_FILE)